# Adjusting Miner Scoring in the Bittensor Subnet: Climate Zone Analysis  

## Introduction  

In this notebook, we aim to lay the groundwork for implementing a climate-based reward function in the Bittensor subnet. This involves mapping grid locations to climate types, analyzing prediction difficulty for each climate zone, and setting up the framework for dynamic reward adjustments.  

### The Reward Function  

The reward function we use to evaluate miner performance is defined as:  

$$
\text{Reward} = \max\left(0, \frac{z - \text{RMSE}}{z}\right)
$$

Where:  
- \( z \): A value that defines the acceptable error margin for a given task.  
- \( \text{RMSE} \): The root mean squared error of the miner’s prediction compared to the ground truth.  

This reward function works as follows:  
- If the miner's prediction is perfect (\( \text{RMSE} = 0 \)), the reward is 1 (maximum reward).  
- As the \( \text{RMSE} \) increases, the reward decreases proportionally.  
- If \( \text{RMSE} \) exceeds \( z \), the reward becomes 0, incentivizing miners to improve their predictions.  

### Why Adjust \( z \) Values?  

The \( z \) value is crucial because it sets the threshold for acceptable error. A higher \( z \) makes the task easier by allowing more error, while a lower \( z \) makes it harder by narrowing the acceptable margin for error.  

Since climate prediction difficulty varies by region:  
- **Easy Regions**: For example, predicting temperature over large, stable oceans may have low variability, making it easier to forecast accurately. Here, a lower \( z \) value can be used.  
- **Challenging Regions**: Areas like mountains or deserts with significant temperature fluctuations are more difficult to predict. For these regions, a higher z value would compensate for that, while lowering \( z \) encourages miners to invest more effort and resources into improving accuracy. So we can play with that.

### Initially: Using SOTA Models to Calibrate \( z \)  

To ensure the reward function reflects realistic difficulty levels:  
1. We use a state-of-the-art (SOTA) AI climate forecasting model to compute the average RMSE for each climate type.  
2. The observed RMSE values will guide the initial \( z \) values, ensuring that rewards align with the inherent difficulty of predicting each climate zone.  
3. Over time, \( z \) values can be manually adjusted to focus efforts on regions where improved predictions provide the most value.  

---

In this notebook, we will:  
1. Visualize the Köppen-Geiger climate zones to understand the diversity of climate types.  
2. Create a grid-based dictionary for efficient climate type lookup.  
3. Use a SOTA climate forecasting model to evaluate RMSE for each climate zone.  
4. Define initial \( z \) values based on the SOTA model’s performance.  

This step-by-step process ensures a fair, data-driven approach to incentivizing accurate climate forecasting in the Bittensor subnet.


## 1. Köppen-Geiger Climate Zones  

The Köppen-Geiger climate classification system is a widely used method for categorizing global climates based on observed temperature, precipitation, and seasonal patterns. It divides the Earth's surface into distinct climate types, each with a unique combination of letters to denote characteristics such as temperature range, humidity levels, and seasonal behavior.  

![Köppen-Geiger Climate Map](../images/Koeppen-Geiger-climate-classification.jpg "Köppen-Geiger Climate Zones Map")  

### Key Features of the Classification System:  
1. **Primary Categories**: The first letter represents the major climate group:  
   - **A**: Tropical  
   - **B**: Dry  
   - **C**: Temperate  
   - **D**: Cold  
   - **E**: Polar  

2. **Secondary Categories**: The second letter adds details about precipitation patterns:  
   - **f**: Fully humid  
   - **w**: Winter dry season  
   - **s**: Summer dry season  
   - **m**: Monsoonal  

3. **Tertiary Categories**: The third letter describes temperature variations:  
   - **a**: Hot summer  
   - **b**: Warm summer  
   - **c**: Cool summer  
   - **d**: Extremely cold winter  
   - **h**: Hot arid  
   - **k**: Cold arid  

### Climate Classes as a Basis for \( z \) Values  

In this notebook, we will use all the climate classes visualized in the Köppen-Geiger map as the basis for defining \( z \) values. Each class will have its own \( z \) value, reflecting the prediction difficulty for that climate type. For example:  
- **Cfa (Warm temperate, fully humid, hot summer)**: Found in large parts of the southeastern United States, this region is moderately challenging to predict and might have an initial \( z = 3 \).  
- **Dfb (Cold, fully humid, warm summer)**: Found in much of Canada and northern Europe, this zone might require a slightly higher \( z \) due to greater variability.  

### Visualization and Climate Zones  

A visualization of the Köppen-Geiger climate zones will help us understand the spatial distribution of these classifications. For regions without documented climate data, such as some parts of the oceans, we will assign an **UNKNOWN** class. This ensures comprehensive coverage and enables predictions across all areas of interest.  

### Setting \( z \) Values  

The exact \( z \) values for each climate type will be calculated later in this notebook, using a SOTA climate forecasting model to establish a baseline RMSE for each class. These \( z \) values will ensure that rewards in the Bittensor subnet are appropriately scaled to reflect the inherent difficulty of making accurate predictions in each climate zone.  


## 2. Create a grid-based dictionary for efficient climate type lookup.  
Since the data source that we will use contain a grid with some pin points on the earth of locations in terms of lat lon with 0.25 degree steps we will create a dictionary lookup where we can see for each of these points on earth what the corresponding climates are. we will start by showing an example of a datapoint from ERA5 (a grid with corresponding tempratures) and then we will 

In [2]:
# lets show a simple example of a datapoint
import sys
sys.path.append('../../')
from climate.data.era5_loader import ERA5DataLoader
import numpy as np

x = ERA5DataLoader()

lat_start = np.random.uniform(x.lat_range[0], x.lat_range[1] - x.area_sample_range[1])
lat_end = lat_start + np.random.uniform(*x.area_sample_range)
lon_start = np.random.uniform(x.lon_range[0], x.lon_range[1] - x.area_sample_range[1])
lon_end = lon_start + np.random.uniform(*x.area_sample_range)

start_time, end_time, predict_hours = x._sample_time_range()

data = x.get_data(
    lat_start=lat_start, 
    lat_end=lat_end, 
    lon_start=lon_start, 
    lon_end=lon_end, 
    start_time=start_time, 
    end_time=end_time
)

# Extract latitude and longitude
lat_lon_points = data[..., :2]  # Select only the latitude and longitude
unique_lat_lon = lat_lon_points.view(-1, 2).unique(dim=0)  # Flatten and get unique points

# Convert to a list of tuples if desired
unique_lat_lon_list = [tuple(point.tolist()) for point in unique_lat_lon]

print("Unique latitude and longitude points:")
print(unique_lat_lon_list)

Unique latitude and longitude points:
[(7.5, -94.75), (7.5, -94.5), (7.5, -94.25), (7.5, -94.0), (7.5, -93.75), (7.5, -93.5), (7.5, -93.25), (7.5, -93.0), (7.5, -92.75), (7.75, -94.75), (7.75, -94.5), (7.75, -94.25), (7.75, -94.0), (7.75, -93.75), (7.75, -93.5), (7.75, -93.25), (7.75, -93.0), (7.75, -92.75), (8.0, -94.75), (8.0, -94.5), (8.0, -94.25), (8.0, -94.0), (8.0, -93.75), (8.0, -93.5), (8.0, -93.25), (8.0, -93.0), (8.0, -92.75), (8.25, -94.75), (8.25, -94.5), (8.25, -94.25), (8.25, -94.0), (8.25, -93.75), (8.25, -93.5), (8.25, -93.25), (8.25, -93.0), (8.25, -92.75), (8.5, -94.75), (8.5, -94.5), (8.5, -94.25), (8.5, -94.0), (8.5, -93.75), (8.5, -93.5), (8.5, -93.25), (8.5, -93.0), (8.5, -92.75), (8.75, -94.75), (8.75, -94.5), (8.75, -94.25), (8.75, -94.0), (8.75, -93.75), (8.75, -93.5), (8.75, -93.25), (8.75, -93.0), (8.75, -92.75), (9.0, -94.75), (9.0, -94.5), (9.0, -94.25), (9.0, -94.0), (9.0, -93.75), (9.0, -93.5), (9.0, -93.25), (9.0, -93.0), (9.0, -92.75), (9.25, -94.75), (

As shown above, the latitude and longitude points are spaced 0.25 degrees apart, so we will need to create a climate dictionary for each pair of latitude and longitude, with a 0.25-degree interval between the points.

In [24]:
def load_koeppen_geiger_data(file_path="/root/ClimateAI/climate/data/Koeppen-Geiger-ASCII.txt"):
    """Load climate data from the Koeppen-Geiger-ASCII.txt file into a dictionary."""
    all_lats = np.arange(-89.75, 90.25, 0.5)
    all_lons = np.arange(-179.75, 180.25, 0.5)
    climate_data = dict()
    for lat in all_lats:
        for lon in all_lons:
            if float(lat) not in climate_data:
                climate_data[float(lat)] = {}

            # initially set all classes to UNKNOWN
            climate_data[float(lat)][float(lon)] = "UNKNOWN"  

    counter = 0
    
    with open(file_path, "r") as file:
        # Skip the header
        next(file)
        
        # Process each line in the file
        for line in file:
            parts = line.split()
            if len(parts) == 3:  # Ensure valid data line
                lat, lon, cls = parts
                lat = float(lat)
                lon = float(lon)

                # Update the climate class
                climate_data[lat][lon] = cls
                counter += 1

    print(f"Loaded {counter} climate data points.")
    return climate_data

koeppen_geiger_data = load_koeppen_geiger_data()

Loaded 92416 climate data points.


In [18]:
list(koeppen_geiger_data.keys())[:5]  # Show the first 5 keys

[-89.75, -89.25, -88.75, -88.25, -87.75]

The Köppen-Geiger data from this source is initially provided with a grid resolution of 0.5 degrees. To increase the resolution to 0.25 degrees and make it suitable for our use case, we will generate a new Köppen-Geiger grid with 0.25-degree intervals. For each new point in this refined grid, we will assign the climate class based on the most frequent class found in the surrounding area. The following Python code can help achieve this by interpolating the original grid data to a finer resolution and assigning the dominant class within the neighborhood of each new point.

In [58]:
from collections import defaultdict

def refine_grid(original_grid, step=0.25):
    """
    Refine a grid to a finer resolution with interpolation.
    
    Args:
        original_grid (dict): Original grid as a dictionary of dictionaries.
        step (float): Step size for the new grid.
    
    Returns:
        dict: New dictionary with refined grid.
    """
    def combine_values(weights, values):
        """Combine values proportionally based on weights."""
        counts = defaultdict(float)
        for weight, value in zip(weights, values):
            counts[value] += weight
        return max(counts, key=counts.get)  # Return the value with the highest weight
    
    # Extract latitude and longitude keys
    lat_keys = sorted(original_grid.keys())
    lon_keys = sorted(next(iter(original_grid.values())).keys())
    
    # Create refined latitude and longitude ranges
    refined_lats = [round(i * step, 2) for i in range(int(lat_keys[0] / step), int(lat_keys[-1] / step) + 1)]
    refined_lons = [round(i * step, 2) for i in range(int(lon_keys[0] / step), int(lon_keys[-1] / step) + 1)]
    
    # Initialize the refined grid
    refined_grid = defaultdict(dict)
    
    # Interpolate for each refined latitude and longitude
    for lat in refined_lats:
        for lon in refined_lons:
            # Find surrounding keys for latitude and longitude
            lat0 = max([k for k in lat_keys if k <= lat], default=min(lat_keys))
            lat1 = min([k for k in lat_keys if k >= lat], default=max(lat_keys))
            lon0 = max([k for k in lon_keys if k <= lon], default=min(lon_keys))
            lon1 = min([k for k in lon_keys if k >= lon], default=max(lon_keys))

            # Get weights for interpolation
            w_lat0 = 1 - (lat - lat0) / (lat1 - lat0) if lat1 != lat0 else 1
            w_lat1 = 1 - w_lat0
            w_lon0 = 1 - (lon - lon0) / (lon1 - lon0) if lon1 != lon0 else 1
            w_lon1 = 1 - w_lon0

            # Collect surrounding values and weights
            weights = [w_lat0 * w_lon0, w_lat0 * w_lon1, w_lat1 * w_lon0, w_lat1 * w_lon1]
            values = [
                original_grid[lat0][lon0], original_grid[lat0][lon1],
                original_grid[lat1][lon0], original_grid[lat1][lon1]
            ]

            # Compute the combined value
            refined_grid[lat][lon] = combine_values(weights, values)
    
    # Convert defaultdict to a standard dictionary
    return {k: dict(v) for k, v in refined_grid.items()}


In [59]:
koeppen_grid_025 = refine_grid(koeppen_geiger_data, step=0.25)

In [4]:
from climate.validator.constants import (
    MAINNET_UID,
    DIFFICULTIES,
)

In [60]:
import pickle

with open("../data/koeppen_geiger_climate_grid_025.pkl", "wb") as f:
    pickle.dump(koeppen_grid_025, f)

In [6]:
x = ERA5DataLoader()

In [7]:
import torch
from typing import List, Tuple

def get_unique_lat_lon(sample_input_data: torch.Tensor) -> List[Tuple[float, float]]:
    # Extract latitude and longitude
    lat_lon_points = data[..., :2]  # Select only the latitude and longitude
    unique_lat_lon = lat_lon_points.view(-1, 2).unique(dim=0)  # Flatten and get unique points

    # Convert to a list of tuples if desired
    unique_lat_lon_list = [tuple(point.tolist()) for point in unique_lat_lon]

    return unique_lat_lon_list

In [8]:
import pickle

with open("/root/ClimateAI/climate/data/koeppen_geiger_climate_grid_025.pkl", "rb") as f:
    koeppen_geiger_climate_grid_025 = pickle.load(f)

In [9]:
def count_climate_types_grid(grid_lats_lons: List[Tuple[float, float]], climate_grid: dict) -> dict:
    climate_counts = dict()
    for lat, lon in grid_lats_lons:
        climate_counts[climate_grid[lat][lon]] = climate_counts.get(climate_grid[lat][lon], 0) + 1
    return climate_counts

In [20]:
sample = x.get_random_sample()
data = sample.input_data

grid_lats_lons = get_unique_lat_lon(data)

climate_counts_grid = count_climate_types_grid(grid_lats_lons, koeppen_geiger_climate_grid_025)

In [21]:
climate_counts_grid

{'ET': 52, 'UNKNOWN': 20}

In [23]:
def get_normalized_difficulty(climate_counts: dict, difficulties: dict) -> float:
    total_difficulty = 0
    total_grid_points = 0
    for climate in climate_counts:
        total_difficulty += difficulties[climate] * climate_counts[climate]
        total_grid_points += climate_counts[climate]

    return total_difficulty / total_grid_points

In [24]:
get_normalized_difficulty(climate_counts_grid, DIFFICULTIES)

1.7222222222222223

In [ ]:
def get_climate_difficulty(climate_counts_grid: dict) -> float:
    # Compute the difficulty based on the climate counts
    # The difficulty is defined as the fraction of the most common climate type
    total_points = sum(climate_counts_grid.values())
    max_count = max(climate_counts_grid.values())
    return max_count / total_points